In [1]:
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
#读取文件  
import os 
import csv
for root, dirs, files in os.walk('dataset'):  
        print(root) #当前目录路径  
        print(files) #当前路径下所有非目录子文件
#进入dataset文件
os.chdir('dataset')

dataset
['ant-1.3.csv', 'ant-1.4.csv', 'ant-1.5.csv', 'ant-1.6.csv', 'ant-1.7.csv', 'camel-1.0.csv', 'camel-1.2.csv', 'camel-1.4.csv', 'camel-1.6.csv', 'ivy-1.1.csv', 'ivy-1.4.csv', 'ivy-2.0.csv', 'jedit-3.2.csv', 'jedit-4.0.csv', 'jedit-4.1.csv', 'jedit-4.2.csv', 'jedit-4.3.csv', 'log4j-1.0.csv', 'log4j-1.1.csv', 'log4j-1.2.csv', 'lucene-2.0.csv', 'lucene-2.2.csv', 'lucene-2.4.csv', 'poi-1.5.csv', 'poi-2.0.csv', 'poi-2.5.csv', 'poi-3.0.csv', 'synapse-1.0.csv', 'synapse-1.1.csv', 'synapse-1.2.csv', 'velocity-1.4.csv', 'velocity-1.5.csv', 'velocity-1.6.csv', 'xalan-2.4.csv', 'xalan-2.5.csv', 'xalan-2.6.csv', 'xalan-2.7.csv', 'xerces-1.2.csv', 'xerces-1.3.csv', 'xerces-1.4.csv']


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import sklearn.model_selection as sk_model_selection
from hyperopt import fmin,tpe,hp,partial,anneal,rand
from sklearn import metrics
from hyperopt.early_stop import no_progress_loss

In [3]:
#写入文件函数
def writercsv(file_path,param,G):
    if os.path.isfile(file_path):
        with open(file_path,'a',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(G)
    else:
        with open(file_path,'w',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(param)
            csv_write.writerow(G)
#文件路径
file_path='..\Anneal_DT\Anneal_DT(5).csv'
param=['Precision','Recall','F-measure','AUC','min_samples_split','min_samples_leaf','max_features','criterion',
       'MSE_Default','MSE_TPE','Bias_Default','Bias_TPE','var_Default','var_TPE','Time']

In [4]:
import time
#顺序读取文件进行实验
num=0
while num<39:
    #读取文件
    df1=pd.read_csv(files[num],header=0)
    df2=pd.read_csv(files[num+1],header=0)
    print(files[num],"->",files[num+1])
    #调参实验
    x_train=df1.iloc[:,0:19].values
    y_train=df1.iloc[:,20].values
    x_test=df2.iloc[:,0:19].values
    y_true=df2.iloc[:,20].values   
    
    #参数空间
    space={'min_samples_split':hp.uniform('min_samples_split',0.1,1.0),
       'min_samples_leaf':hp.uniform('min_samples_leaf',0.1,1.0),
       'max_features':hp.choice('max_features',['sqrt','log2']),
      'criterion':hp.choice('criterion',['gini','entropy'])}
    #代理函数
    algo=partial(tpe.suggest)
    #algo=partial(anneal.suggest)
    #algo=partial(rand.suggest)
    def DT(args):
        clf=tree.DecisionTreeClassifier(random_state=0,
                                    min_samples_split=float(args['min_samples_split']),
                                     min_samples_leaf=float(args['min_samples_leaf']),
                                     max_features=args['max_features'],
                                     criterion=args['criterion'])
        auc=sk_model_selection.cross_val_score(clf,x_train,y_train,scoring='roc_auc',cv=10)
        return -auc.mean()
    time_start = time.process_time()
    #early_stop_fn = no_progress_loss(50)早停
    best=fmin(DT,space=space,algo=algo,max_evals=200)
    time_end = time.process_time()  # 记录结束时间
    time_sum = time_end - time_start  # 计算的时间差为程序的执行时间，单位为秒/s
    #设置离散变量
    if best['max_features']==0:
        best['max_features']='sqrt'
    else: 
        best['max_features']='log2'
    if best['criterion']==0:
        best['criterion']='gini'
    else:
        best['criterion']='entropy'
    #参数带回实验进行预测
    model=tree.DecisionTreeClassifier(random_state=0,min_samples_split=float(best['min_samples_split']),
                                 min_samples_leaf=float(best['min_samples_leaf']),
                                 max_features=best['max_features'],
                                 criterion=best['criterion'])
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    #评分
    G=[]
    probas=model.predict_proba(x_test)
    fpr,tpr,thresholds=roc_curve(y_true,probas[:,1])
    auc=metrics.auc(fpr,tpr)
    f_measure=metrics.f1_score(y_true,y_pred,average='weighted')
    recall=metrics.recall_score(y_true,y_pred,average='weighted')
    precision=metrics.precision_score(y_true,y_pred,average='weighted')
    G.append(precision)
    G.append(recall)
    G.append(f_measure)
    G.append(auc)
    G.append(best['min_samples_split'])
    G.append(best['min_samples_leaf'])
    G.append(best['max_features'])
    G.append(best['criterion'])
    #计算模型稳定性
    model_d=tree.DecisionTreeClassifier(random_state=0)
    model_h=tree.DecisionTreeClassifier(random_state=0,min_samples_split=float(best['min_samples_split']),
                                 min_samples_leaf=float(best['min_samples_leaf']),
                                 max_features=best['max_features'],
                                 criterion=best['criterion'])    
    #bias and variance 
    from mlxtend.evaluate import bias_variance_decomp
    Expected_loss_d,Bias_Default,var_Default=bias_variance_decomp(model_d,x_train,y_train,x_test,y_true,
                                  loss='mse',num_rounds=100,random_seed=1)
    Expected_loss_TPE,Bias_TPE,var_TPE=bias_variance_decomp(model_h,x_train,y_train,x_test,y_true,
                                  loss='mse',num_rounds=100,random_seed=1)
    G.append(Expected_loss_d)
    G.append(Expected_loss_TPE)
    G.append(Bias_Default)
    G.append(Bias_TPE)
    G.append(var_Default)
    G.append(var_TPE)
    #记录时间
    G.append(time_sum)
    #写入文件
    writercsv(file_path,param,G)
    print('写入文件成功')
    #版本变化
    num=num+1
    print(num)
    #项目变化
    if((num==4) or (num==8)or(num==11)or(num==16)or(num==19)or(num==22)or(num==26)or(num==29)or(num==32)or(num==36)):
        num=num+1
        print("项目变化",num)

ant-1.3.csv -> ant-1.4.csv
100%|█████████████████████████████████████████████| 200/200 [00:02<00:00, 67.87trial/s, best loss: -0.8395454545454546]
写入文件成功
1
ant-1.4.csv -> ant-1.5.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 66.09trial/s, best loss: -0.6533241758241759]
写入文件成功
2
ant-1.5.csv -> ant-1.6.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 64.33trial/s, best loss: -0.7197649572649573]
写入文件成功
3
ant-1.6.csv -> ant-1.7.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 59.36trial/s, best loss: -0.7905888888888889]
写入文件成功
4
项目变化 5
camel-1.0.csv -> camel-1.2.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 63.69trial/s, best loss: -0.7091856060606061]
写入文件成功
6
camel-1.2.csv -> camel-1.4.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 56.56trial/s, best loss: -0.5641622266622266]
写入文件成功
7
camel-1.4.csv -> camel-1.6.csv
100%|█████████████